# Here comes the sun
> When does the sun rise and set? I have a fuzzy idea about how this works, here I want to make it a little bit less fuzzy.

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [math]
- image: https://exitoina.uol.com.br/media/_versions/beatlessss_widelg.jpg
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

## Using suntime package
Eventhough this could be cheating, it's a good starting point to get a feel...
This example is taken straight from the documentation:

In [22]:
#collapse
import datetime
from suntime import Sun, SunTimeException

latitude = 51.21
longitude = 21.01

sun = Sun(latitude, longitude)

# Get today's sunrise and sunset in UTC
today_sr = sun.get_sunrise_time()
today_ss = sun.get_sunset_time()
print('Today ({}) at Warsaw the sun raised at {} and get down at {} UTC'.
      format(datetime.datetime.today().strftime('%Y-%m-%d'), today_sr.strftime('%H:%M'), today_ss.strftime('%H:%M')))

# On a special date in your machine's local time zone
abd = datetime.date(2014, 10, 3)
abd_sr = sun.get_local_sunrise_time(abd)
abd_ss = sun.get_local_sunset_time(abd)
print('On {} the sun at Warsaw raised at {} and get down at {}.'.
      format(abd, abd_sr.strftime('%H:%M'), abd_ss.strftime('%H:%M')))

# Error handling (no sunset or sunrise on given location)
latitude = 87.55
longitude = 0.1
sun = Sun(latitude, longitude)
try:
    abd_sr = sun.get_local_sunrise_time(abd)
    abd_ss = sun.get_local_sunset_time(abd)
    print('On {} at somewhere in the north the sun raised at {} and get down at {}.'.
          format(abd, abd_sr.strftime('%H:%M'), abd_ss.strftime('%H:%M')))
except SunTimeException as e:
    print("Error: {0}.".format(e))

Today (2020-11-02) at Warsaw the sun raised at 05:32 and get down at 15:07 UTC
On 2014-10-03 the sun at Warsaw raised at 06:39 and get down at 18:10.
Error: The sun never rises on this location (on the specified date).


I currently live here: 58°04'00.8"N 11°42'10.7"E

This one can be converted to [Decimal degrees](https://en.wikipedia.org/wiki/Decimal_degrees).

In [37]:
def longitude_str_to_num(s:str):
    """
    Convert a longitude or latitude in the following format : 58°04'00.8"N
    to degrees
    """
    
    s1,s_=s.split('°')
    s2,s_=s_.split("'")
    s3,s_=s_.split('"')
    
    
    D = float(s1)
    M = float(s2)
    S = float(s3)
    
    D_deg = D + M/60 + S/3600
    
    if (s_=='W') or (s_=='S'):
        D_deg*=-1
    
    
    return D_deg

In [38]:
latitude = longitude_str_to_num('58°04\'00.8"N')
latitude

58.06688888888889

In [39]:
longitude = longitude_str_to_num('11°42\'10.7"E')
longitude

11.702972222222222

In [40]:
sun = Sun(latitude, longitude)

In [59]:
sunrise_time = sun.get_sunrise_time()
sunrise_time

datetime.datetime(2020, 11, 2, 6, 31, tzinfo=tzutc())

In [60]:
sunset_time = sun.get_sunset_time()
sunset_time

datetime.datetime(2020, 11, 2, 15, 21, tzinfo=tzutc())

Getting these times in local time zone is somewhat messy:

In [65]:
import pytz
timezone = pytz.timezone(r'Europe/Stockholm')
sunrise_time.replace(tzinfo=pytz.utc).astimezone(timezone)

datetime.datetime(2020, 11, 2, 7, 31, tzinfo=<DstTzInfo 'Europe/Stockholm' CET+1:00:00 STD>)

## Own implementation

The [Sun rise equation](https://en.wikipedia.org/wiki/Sunrise_equation) is written:
$$\cos \omega_{\circ}=-\tan \phi \times \tan \delta$$
where $\phi$ is the latitude and $\delta$ is the earth inclination angle to the sun, which changes over the seasons.

The [declination-angle](https://www.pveducation.org/pvcdrom/properties-of-sunlight/declination-angle) can be calculated:
$$ \delta=-23.45^{\circ} \times \cos \left(\frac{360}{365} \times(d+10)\right) $$
where $d=1$ at january 1st.

Putting it all together:

In [76]:
import numpy as np

def calculate_declination_angle(day):
    delta_deg = -23.45*np.cos(360/365*(day+10))
    delta = np.deg2rad(delta_deg)    
    return delta

def sun_rise_time(latitude:float, day:int):
    """
    param: latitude 
    param: day, january 1 --> day=1
    """
    delta = calculate_declination_angle(day=day)
    phi=latitude
    omega0 = np.arccos(-np.tan(phi)*np.tan(delta))
    
    return omega0
    

In [77]:
day=1
calculate_declination_angle(day=day)

0.05964773709497602